In [1]:
import zmq
import time
import socket
import os
import sys
import logging
import threading
import warnings
import numpy as np
from numpy import matlib
from __future__ import division

# setup the logger
logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)

# Check wich computer to decide where the things are mounted
comp_name=socket.gethostname()
logger.info('Computer: ' + comp_name)

if 'passaro' in comp_name:
    repos_folder = os.path.abspath('/mnt/cube/earneodo/repos')
    experiment_folder = os.path.join('/mnt/cube/earneodo/bci_zf/')
    rec_folder = os.path.join('/usr/local/experiment/raw_data')
    

if 'lookfar' in comp_name:
    repos_folder = os.path.abspath('/Users/zeke/repos')
    experiment_folder = os.path.join('/Users/zeke/bci_zf')
    rec_folder = os.path.join('/Users/zeke/bci_zf/raw_data')

sys.path.append(os.path.join(repos_folder, 'soundflow', 'sound_tools'))
sys.path.append(os.path.join(repos_folder, 'ephysflow'))

from file_tools import experiment as et
from rig_tools import open_ephys as oe, beagle_bone as bb

2017-12-08 20:02:19,765 root         INFO     Computer: passaro


### Session tools and methods

In [2]:
# command functions
def cmd_play_wav(wav_file):
    cmd_line = 'play_wav stim_file ' + wav_file
    return cmd_line

def cmd_trial_pin(on):
    #on is boolean
    return 'trial_pin on {}'.format(on)

def cmd_trial_number(tr_num):
    #on is boolean
    return 'trial_number number {}'.format(int(tr_num))
        
def run_trial(rig_par={'bb': None, 'oe':None}, 
              trial_par={'iti':500., 'tr_num': 5, 'wave_file': None}):
    time.sleep(int(trial_par['iti']*100)/100000.)
    rig_par['bb'].send_command(cmd_trial_pin(True))
    rig_par['oe'].send_command(cmd_trial_number(trial_par['tr_num']))
    rig_par['bb'].send_command(cmd_trial_number(trial_par['tr_num']))
    rig_par['oe'].send_command(cmd_play_wav(trial_par['stim']))
    rig_par['bb'].send_command(cmd_play_wav(trial_par['stim']))
    rig_par['bb'].send_command(cmd_trial_pin(False))
    
class Recorder(threading.Thread):
    def __init__(self, rig_par, control_Event, new=False, rec_par={'CreateNewDir': '0'}, restart=True, 
                 group=None, target=None, name=None, 
                 args=(), kwargs=None, verbose=None):
        threading.Thread.__init__(self, group=group, target=target, name=name, verbose=verbose)
        
        logging.info('Initializing recorder')
        self.name = 'Recorder'
        self.rig = rig_par
        self.rec_sys = rig_par['rec']
        self.recording = threading.Event() #Event
        self.control_signal = control_Event # Event
        self.recording_path = None
        self.rec_par = rec_par
        self.new_rec = new
        self.force_restart = restart
        self.recording.clear()
        
   
    def run(self):
        logging.debug('recording')
        # If starting of a new recorded is forced
        ok_to_start = False
        
        if self.is_recording():
            if self.force_restart:
                self.stop_record()
                ok_to_start = True
                
        if self.control_signal is not None:
            while not self.control_signal.is_set():
                pass
            logging.debug('Start record signal received')
            self.start_record(new=self.new_rec, rec_par=self.rec_par)
            while self.control_signal.is_set():
                if self.recording.is_set():
                    pass
                else:
                    logging.info('Stop recording Forced')
                    self.control_signal.clear()
                    break
            logging.debug('Stop record signal received')
            self.stop_record()

    def start_record(self, new=False, rec_par={'CreateNewDir':'0'}):
        if self.rec_sys.start_rec(rec_par=rec_par):
            logging.debug('Started recording')
            self.recording_path = self.rec_sys.get_rec_path()
            self.recording.set()
        return self.recording_path
                    
    def stop_record(self, ):
        if self.rec_sys.query_status('Recording'):
            self.rec_sys.stop_rec()
            self.recording.clear()
            logging.info('Stopped recording')
            
    def break_record(self, ):
        self.rec_sys.break_rec()
            
    def signal_stop(self):
        self.recording.clear()
        
    def is_recording(self):
        return self.rec_sys.query_status(status_query='Recording')
    
    def report_recording(self):
        return self.recording.is_set()
    
    def report_recording_parh(self):
        return self.recording_path()
    
    def get_rec_path(self):
        return self.rec_sys.get_rec_path()
    
class Block:
    def __init__(self, stim_list, iti_bounds_ms, size, random = True):
        self.trials = 0
        self.iti_bounds = iti_bounds_ms
        self.stim_list = stim_list
        self.size = size
        self.trial_pars = {'iti': None, 'tr_num': 0, 'stim': None}
        self.stim_order = matlib.repmat(np.arange(len(stim_list)),
                                           int(np.ceil(size/len(stim_list))),
                                           1)[:size].flatten()
        if random:
            np.random.shuffle(self.stim_order)
    
    def __iter__(self,):
        return self
    
    def next(self, ):
        if self.trials < self.size:
            self.trial_pars['iti'] = np.random.randint(self.iti_bounds[0], self.iti_bounds[1])
            #print self.trials
            self.trial_pars['tr_num'] = self.trials
            self.trial_pars['stim'] = self.stim_list[self.stim_order[self.trials]]
            self.trials+=1
            return self.trial_pars
        else:
            raise StopIteration()
            
class Runner(threading.Thread):
    def __init__(self, block, rig_par, recorder=None, group=None, target=None, name=None, 
                 args=(), kwargs=None, verbose=None):
        threading.Thread.__init__(self, group=group, target=target, name=name, verbose=verbose)
        
        logging.info('Initializing block')
        self.block = block #block object
        self.rig = rig_par
        self.stim_sys = rig_par['bb']
        self.rec_sys = rig_par['oe']
        self.running = threading.Event() #Event
        self.finished = False
        self.trial = None
        self.running.clear()
        self.recorder = recorder #Record control
   
    def run(self):
        logging.debug('running block')
        if not self.running.is_set():
            print 'Starting Block'
#             if self.is_recording()==True:
#                 logging.debug('Starting recording')
#                 self.recorder.start()
#                 time.sleep(2)
            self.running.set()
            while True:
                try:
                    if self.running.is_set():
                        self.trial = self.block.next()
                        logging.info('Running trial {}'.format(self.trial))
                        print self.trial
                        run_trial(rig_par=self.rig, trial_par=self.trial)
                    else:
                        self.end_block()
                        break

                except StopIteration:
                    self.finished = True
                    self.running.clear()
                    print 'finished block'
                    break
            
            if self.is_recording()==True:
                print('Runner stopping recording')
                self.recorder.signal_stop()
        else:
            logging.info('Cant start, already running')
            
        return self.finished
                    
    def end_block(self):
        print('stopping the block in trial {}'.format(self.trial['tr_num']))
        self.finished = False
        
    def signal_stop(self):
        self.running.clear()
        
    def is_running(self):
        return self.running.is_set()
    
    def is_recording(self):
        logging.debug('runner checking recording status')
        logging.debug('{}'.format(self.recorder))
        if self.recorder is not None:
            logging.debug('status {}'.format(self.recorder.report_recording()))
            return self.recorder.report_recording()
        else:
            return None

class H5er():
    # To do: a thread to log the input/output of every trial, 
    # that is init by a session and passed to a runner
    def __init__(self):
        raise NotImplementedError
        
class Session:
    def __init__(self, ephys, beagle, bird_id='z000', depth=100, descr=None):
        self.ephys = ephys
        self.beagle = beagle
        self.bird_folder = None
        self.sess_folder = None
        self.bird_id = bird_id
        self.electrode_depth = depth #append to rec folder
        self.experiment_descr = descr #prepend to rec folder
        self.block_runner = None
        self.trial_number = 1
        self.make_bird_folder()
        self.recording = threading.Event()
        self.record_control = threading.Event()
        self.recorder = Recorder({'rec': self.ephys}, self.record_control)
        self.recording.clear()
        self.record_control.clear()
        self.timers = {}
        
    def make_bird_folder(self, ):
        self.bird_folder = os.path.join(rec_folder, self.bird_id)
        et.mkdir_p(self.bird_folder)
    
    def start_record(self, duration=0, new=False, force_restart=False, max_rec_size=0):
        if new:
            rec_par = {'CreateNewDir': '1', 
                      'RecDir': self.bird_folder,
                      'AppendText': str(self.electrode_depth),
                      'PrependText': self.experiment_descr}
        else:
            rec_par = {'CreateNewDir':'0'}
        
        if self.is_recording():
            logger.info('Trying to record but already recording')
        
        else:
            self.record_control.set()
            self.recorder = Recorder({'rec': self.ephys}, self.record_control,
                                    new=new, rec_par=rec_par, restart=force_restart)
            self.recorder.start()
            if self.recorder.report_recording():
                logger.debug('Started recording')
                self.sess_folder = self.recorder.report_recording_path()
                self.recording.set()
                
            if duration > 0:
                self.timers['stop_record'] = threading.Timer(duration, self.stop_record)
                self.timers['stop_record'].start()
            
            if max_rec_size > 0:
                if duration==0:
                    warnings.warn('Cannot chunk an unlimited recording for now', RuntimeWarning)
                break_points = np.arange(0, duration, max_rec_size)[1:]
                self.timers['break_record'] = []
                for t_b in break_points:
                    logger.info('setting break timer at {}'.format(t_b))
                    break_timer = threading.Timer(t_b, self.break_record)
                    self.timers['break_record'].append(break_timer)
                    break_timer.start()
                    
    def stop_record(self, ):
        if 'stop_record' in self.timers:
            self.timers['stop_record'].cancel()
            del(self.timers['stop_record'])
        if 'break_record' in self.timers:
            [tm.cancel() for tm in self.timers['break_record']]
            del(self.timers['break_record'])
            
        if self.recorder.report_recording():
            self.recorder.signal_stop()
            self.recording.clear()
            logging.info('Stopped recording')
            
    def break_record(self, ):
        if self.recorder.report_recording():
            logger.debug('Was recording')
            logger.info('Inserting a break in the recording')
            self.recorder.break_record()
        
    def start_block(self, block, record=False, new=False):
        if record:
            logging.info('Starting to record')
            self.start_record(new=True)
            time.sleep(2)
            
        logger.info('Starting block')
        self.block_runner = Runner(block, {'bb': self.beagle, 'oe': self.ephys}, 
                                   recorder=self.recorder)
        self.block_runner.start()
        
    def monitor_block(self, ):
        if self.is_recording():
            self.stop_record()
    
    def stop_block(self):
        self.block_runner.signal_stop()
        if self.is_recording:
            self.stop_record()
        
    def is_recording(self):
        return self.recording.is_set()
    
    def is_running(self):
        return self.block_runner.is_running()

In [3]:
# open the devices
oe_zmq = oe.OpenEphysEvents(port='5556', ip='127.0.0.1')
bb_zmq = bb.BeagleBone(port='5559', ip='tsippor.ucsd.edu')

bb_zmq.connect()
oe_zmq.connect()

In [4]:
oe_zmq.stop_acq()
oe_zmq.start_acq()

Acquistion stopped
Acquisition Started


# just record for some time, creating a new record every some seconds

In [5]:
max_rec_size = 1800
total_rec = 3600*3

ss = Session(oe_zmq, bb_zmq, bird_id='z049', depth=2000, descr='awake')
ss.start_record(total_rec, new=True, force_restart=True, max_rec_size=max_rec_size)

2017-12-08 16:36:09,178 root         INFO     Initializing recorder
2017-12-08 16:36:09,181 root         INFO     Initializing recorder
2017-12-08 16:36:09,183 root         INFO     setting break timer at 1800
2017-12-08 16:36:09,186 root         INFO     setting break timer at 3600
2017-12-08 16:36:09,188 root         INFO     setting break timer at 5400
2017-12-08 16:36:09,190 root         INFO     setting break timer at 7200
2017-12-08 16:36:09,191 root         INFO     setting break timer at 9000


OK to start
Recording path: /usr/local/experiment/raw_data/z049/awake_2017-12-08_16-36-09_2000


2017-12-08 17:06:09,187 root         INFO     Inserting a break in the recording


Breaking recording in progress


2017-12-08 17:36:09,191 root         INFO     Inserting a break in the recording


Breaking recording in progress


2017-12-08 18:06:09,192 root         INFO     Inserting a break in the recording


Breaking recording in progress


2017-12-08 18:36:09,194 root         INFO     Inserting a break in the recording


Breaking recording in progress


In [6]:
ss.break_record()

2017-12-08 12:59:16,149 root         INFO     Inserting a break in the recording


Breaking recording in progress


2017-12-08 13:29:05,231 root         INFO     Inserting a break in the recording


Breaking recording in progress


2017-12-08 13:59:05,231 root         INFO     Inserting a break in the recording


...

Breaking recording in progress


2017-12-08 15:29:05,252 root         INFO     Inserting a break in the recording


Breaking recording in progress


2017-12-08 15:59:05,228 root         INFO     Stopped recording
2017-12-08 15:59:05,228 root         INFO     Stop recording Forced
2017-12-08 15:59:05,239 root         INFO     Stopped recording


Recording stopped


In [6]:
ss.stop_record()

2017-12-08 19:00:01,426 root         INFO     Stopped recording
2017-12-08 19:00:01,426 root         INFO     Stop recording Forced
2017-12-08 19:00:01,437 root         INFO     Stopped recording


Recording stopped


### Make recordings with stimulus

In [18]:
# Bird specific data
bird_id = 'z049'
electrode_depth = 2000 #append to rec folder
experiment_descr = 'asleep' #prepend to rec folder

wave_files = [os.path.abspath('/root/experiment/stim/bos_tag.wav'),
              os.path.abspath('/root/experiment/stim/bos_rev_tag.wav'),
              os.path.abspath('/root/experiment/stim/con_tag.wav'),
              os.path.abspath('/root/experiment/stim/bos_syn_tag.wav')]

# wave_files = [os.path.abspath('/root/experiment/stim/bos_tag.wav'), 
#               os.path.abspath('/root/experiment/stim/bos_rev_tag.wav')]
wave_file = wave_files[0]
cmd_line = 'trial type passive number 1 stim_file ' + wave_file
bb_zmq.send_command(cmd_line)

'ok trial:1, file:/root/experiment/stim/bos_tag.wav'

In [13]:
wave_files

['/root/experiment/stim/bos_tag.wav',
 '/root/experiment/stim/bos_rev_tag.wav',
 '/root/experiment/stim/con_tag.wav',
 '/root/experiment/stim/bos_syn_tag.wav']

In [7]:
waves_to_use = wave_files[:2]

In [8]:
waves_to_use

['/root/experiment/stim/bos_tag.wav', '/root/experiment/stim/bos_rev_tag.wav']

In [9]:
# run the one session
time.sleep(1200)
ss = Session(oe_zmq, bb_zmq, bird_id='z049', depth=2000, descr='asleep_stim')
a_block = Block(waves_to_use, [7000, 15000], 100)
ss.start_block(a_block, record=True, new=True)
#time.sleep(5)
#ss.stop_block()

2017-12-08 20:27:03,259 root         INFO     Initializing recorder
2017-12-08 20:27:03,264 root         INFO     Starting to record
2017-12-08 20:27:03,265 root         INFO     Initializing recorder


OK to start
Recording path: /usr/local/experiment/raw_data/z049/asleep_stim_2017-12-08_20-27-03_2000


2017-12-08 20:27:05,272 root         INFO     Starting block
2017-12-08 20:27:05,287 root         INFO     Initializing block


Starting Block


2017-12-08 20:27:05,299 root         INFO     Running trial {'tr_num': 0, 'stim': '/root/experiment/stim/bos_tag.wav', 'iti': 8963}


...

{'tr_num': 98, 'stim': '/root/experiment/stim/bos_tag.wav', 'iti': 8897}


2017-12-08 20:47:06,019 root         INFO     Running trial {'tr_num': 99, 'stim': '/root/experiment/stim/bos_tag.wav', 'iti': 12014}


{'tr_num': 99, 'stim': '/root/experiment/stim/bos_tag.wav', 'iti': 12014}


2017-12-08 20:47:19,146 root         INFO     Stop recording Forced
2017-12-08 20:47:19,157 root         INFO     Stopped recording


finished block
Runner stopping recording
Recording stopped


In [14]:
ss.stop_block()

2017-10-25 14:44:10,859 root         INFO     Stopped recording
2017-10-25 14:44:10,859 root         INFO     Stop recording Forced
2017-10-25 14:44:10,867 root         INFO     Stopped recording


Recording stopped
stopping the block in trial 1


In [17]:
# run a night experiment
block_trials = 210
block_iti = [2000, 7000]
time.sleep(7200)
#ss = Session(oe_zmq, bb_zmq, bird_id='z023', depth=0, descr='anesth_surf')
blocks = []
for i_block in range(2):
    print "Starting block {}".format(i_block)
    a_block = Block(wave_files, block_iti, block_trials)
    ss.start_block(a_block, record=True, new=True)
    #time.sleep(block_trials*int(np.ceil(block_iti[1]*0.0011)))
    time.sleep(3600)
    ss.stop_block()
    print 'Block {} finished'.format(i_block)
    blocks.append(a_block)
    time.sleep(1)
    

2017-10-24 00:51:25,435 root         INFO     Running trial {'tr_num': 35, 'stim': '/root/experiment/stim/bos_syn_tag.wav', 'iti': 9238}


{'tr_num': 35, 'stim': '/root/experiment/stim/bos_syn_tag.wav', 'iti': 9238}


2017-10-24 00:51:36,014 root         INFO     Running trial {'tr_num': 36, 'stim': '/root/experiment/stim/bos_tag.wav', 'iti': 9925}


{'tr_num': 36, 'stim': '/root/experiment/stim/bos_tag.wav', 'iti': 9925}


2017-10-24 00:51:47,420 root         INFO     Running trial {'tr_num': 37, 'stim': '/root/experiment/stim/bos_rev_tag.wav', 'iti': 6911}


...

{'tr_num': 208, 'stim': '/root/experiment/stim/con_tag.wav', 'iti': 3233}


2017-10-24 04:11:31,400 root         INFO     Running trial {'tr_num': 209, 'stim': '/root/experiment/stim/bos_rev_tag.wav', 'iti': 2227}


{'tr_num': 209, 'stim': '/root/experiment/stim/bos_rev_tag.wav', 'iti': 2227}
finished block

2017-10-24 04:11:35,439 root         INFO     Stop recording Forced
2017-10-24 04:11:35,455 root         INFO     Stopped recording



Runner stopping recording
Recording stopped
Block 1 finished
